In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv("../dataset/cleaned_data.csv")
df.head()


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,category,sub-category,product_name,sales,quantity,discount,profit,order_year,order_month,order_month_name
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136,2016,11,November
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820,2016,11,November
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714,2016,6,June
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310,2015,10,October
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164,2015,10,October


In [4]:
df.shape

(9994, 24)

In [5]:
df.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit', 'order_year',
       'order_month', 'order_month_name'],
      dtype='object')

In [6]:
df[['sales','discount','profit']].describe()

,sales,discount,profit
count,9994.000000,9994.000000,9994.000000
mean,229.858001,0.156203,28.656896
std,623.245101,0.206452,234.260108
min,0.444000,0.000000,-6599.978000
25%,17.280000,0.000000,1.728750
50%,54.490000,0.200000,8.666500
75%,209.940000,0.200000,29.364000
max,22638.480000,0.800000,8399.976000


In [7]:
df['is_loss']=df['profit']<0

In [8]:
df['is_loss'].value_counts(normalize=True)


is_loss
False    0.812788
True     0.187212
Name: proportion, dtype: float64

In [9]:
df['discount_bucket']=pd.cut(
    df['discount'],
    bins=[0,0.1,0.2,0.3,0.4,1],
    labels=['0-10%','10-20%','20-30%','30-40%','40%+'],
    include_lowest=True
)

In [10]:
df[['discount', 'discount_bucket']].head(10)

,discount,discount_bucket
0,0.00,0-10%
1,0.00,0-10%
2,0.00,0-10%
3,0.45,40%+
4,0.20,10-20%
5,0.00,0-10%
6,0.00,0-10%
7,0.20,10-20%
8,0.20,10-20%
9,0.00,0-10%


In [11]:
df[df['sales'] == 0]

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,product_name,sales,quantity,discount,profit,order_year,order_month,order_month_name,is_loss,discount_bucket


In [12]:
df['profit_margin']=df['profit']/df['sales']
df['profit_margin'].describe()

count    9994.000000
mean        0.120314
std         0.466754
min        -2.750000
25%         0.075000
50%         0.270000
75%         0.362500
max         0.500000
Name: profit_margin, dtype: float64

In [13]:
profit_by_dis=(
    df.groupby('discount_bucket')['profit'].sum().sort_index()
)
profit_by_dis

C:\Users\Hp\AppData\Local\Temp\ipykernel_10284\4042049155.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('discount_bucket')['profit'].sum().sort_index()


discount_bucket
0-10%     330016.7802
10-20%     91756.2975
20-30%    -10369.2774
30-40%    -25448.1881
40%+      -99558.5905
Name: profit, dtype: float64

In [14]:
loss_rate=(
    df.groupby('discount_bucket')['is_loss'].mean().sort_index()
)
loss_rate

C:\Users\Hp\AppData\Local\Temp\ipykernel_10284\126439751.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('discount_bucket')['is_loss'].mean().sort_index()


discount_bucket
0-10%     0.000818
10-20%    0.139930
20-30%    0.916300
30-40%    0.888412
40%+      1.000000
Name: is_loss, dtype: float64

In [15]:
discount_summary = (
    df.groupby('discount_bucket')
    .agg(
        total_sales=('sales', 'sum'),
        total_profit=('profit', 'sum'),
        avg_profit_margin=('profit_margin', 'mean'),
        loss_rate=('is_loss', 'mean'),
        order_count=('order_id', 'count')
    )
    .reset_index()
)

discount_summary


C:\Users\Hp\AppData\Local\Temp\ipykernel_10284\1732638462.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('discount_bucket')


,discount_bucket,total_sales,total_profit,avg_profit_margin,loss_rate,order_count
0,0-10%,1.142278e+06,330016.7802,0.336618,0.000818,4892
1,10-20%,7.921529e+05,91756.2975,0.174839,0.139930,3709
2,20-30%,1.032267e+05,-10369.2774,-0.115481,0.916300,227
3,30-40%,1.309112e+05,-25448.1881,-0.216907,0.888412,233
4,40%+,1.286323e+05,-99558.5905,-1.089003,1.000000,933


In [16]:
category_discount = (
    df.groupby(['category', 'discount_bucket'])
    .agg(
        total_profit=('profit', 'sum'),
        loss_rate=('is_loss', 'mean')
    )
    .reset_index()
)

category_discount.head()


C:\Users\Hp\AppData\Local\Temp\ipykernel_10284\3039861206.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['category', 'discount_bucket'])


,category,discount_bucket,total_profit,loss_rate
0,Furniture,0-10%,65244.0883,0.004386
1,Furniture,10-20%,7684.9406,0.274363
2,Furniture,20-30%,-10695.3169,0.932432
3,Furniture,30-40%,-18578.5345,1.000000
4,Furniture,40%+,-25203.9047,1.000000


In [17]:
category_pivot = category_discount.pivot(
    index='category',
    columns='discount_bucket',
    values='total_profit'
)

category_pivot


discount_bucket,0-10%,10-20%,20-30%,30-40%,40%+
category,,,,,
Furniture,65244.0883,7684.9406,-10695.3169,-18578.5345,-25203.9047
Office Supplies,131592.1872,38038.7512,0.0000,0.0000,-47140.1376
Technology,133180.5047,46032.6057,326.0395,-6869.6536,-27214.5482


In [18]:
subcat_loss = (
    df.groupby('sub-category')
    .agg(
        total_profit=('profit', 'sum'),
        loss_rate=('is_loss', 'mean'),
        avg_discount=('discount', 'mean')
    )
    .sort_values(by='total_profit')
)

subcat_loss.head(10)


,total_profit,loss_rate,avg_discount
sub-category,,,
Tables,-17725.4811,0.636364,0.261285
Bookcases,-3472.5560,0.478070,0.211140
Supplies,-1189.0995,0.173684,0.076842
Fasteners,949.5182,0.055300,0.082028
Machines,3384.7569,0.382609,0.306087
Labels,5546.2540,0.000000,0.068681
Art,6527.7870,0.000000,0.074874
Envelopes,6964.1767,0.000000,0.080315
Furnishings,13059.1436,0.174504,0.138349


In [19]:
region_analysis = (
    df.groupby('region')
    .agg(
        total_profit=('profit', 'sum'),
        loss_rate=('is_loss', 'mean'),
        avg_discount=('discount', 'mean')
    )
    .sort_values(by='total_profit')
)

region_analysis


,total_profit,loss_rate,avg_discount
region,,,
Central,39706.3625,0.318984,0.240353
South,46749.4303,0.159877,0.147253
East,91522.7800,0.194171,0.145365
West,108418.4489,0.099282,0.109335


 Key Insights (Draft):-


-High discounts (>30%) show significantly higher loss rates

-Some categories lose money even at low discounts

-Certain sub-categories are consistently unprofitable

-Discounting strategy needs category-specific control

! These are draft insights, not final.

In [20]:
df.to_csv("../dataset/feature_data.csv", index=False)

** Key KPIs**


-Total Sales

-Total Profit

-Overall Profit Margin

-Loss Rate (% orders with loss)

-Average Discount

-Profit by Discount Bucket

In [21]:
kpis = {
    "total_sales": df['sales'].sum(),
    "total_profit": df['profit'].sum(),
    "overall_profit_margin": df['profit'].sum() / df['sales'].sum(),
    "loss_rate": df['is_loss'].mean(),
    "avg_discount": df['discount'].mean()
}

kpis


{'total_sales': np.float64(2297200.8603000003),
 'total_profit': np.float64(286397.0217),
 'overall_profit_margin': np.float64(0.12467217240315603),
 'loss_rate': np.float64(0.18721232739643787),
 'avg_discount': np.float64(0.15620272163297977)}

-Higher discounts (>30%) show significantly higher loss rates

-Some sub-categories are unprofitable even at low discounts

-Blanket discounting hurts profitability

-Losses are concentrated in specific categories/regions

Business Recommendations

Examples:

Cap discounts for loss-prone sub-categories

Use category-based discounting instead of flat discounts

Introduce margin-based discount approval

In [22]:
df.to_csv("../dataset/final_analysis_data.csv", index=False)

In [23]:
df['is_loss'] = df['is_loss'].astype(int)


In [24]:
df['is_loss'].unique()



array([0, 1])

In [27]:
df.to_csv(
    "../dataset/final_data.csv",
    index=False
)


In [28]:
df1 = pd.read_csv("../dataset/final_data.csv")
df1.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit', 'order_year',
       'order_month', 'order_month_name', 'is_loss', 'discount_bucket',
       'profit_margin'],
      dtype='object')